In [17]:
%load_ext autoreload
%autoreload 2

from artfinder import Crossref, load_csv, strict_filter
import pandas as pd
import datetime
import re
import logging

logging.basicConfig(level=logging.INFO)
crosref = Crossref(app='artfinder', email='aapopov1@mephi.ru')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
df = load_csv('database/processed/barcikowski_full.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   publisher               548 non-null    string        
 1   license                 348 non-null    object        
 2   is_referenced_by_count  548 non-null    int64         
 3   link                    486 non-null    object        
 4   authors                 548 non-null    object        
 5   abstract                184 non-null    string        
 6   title                   548 non-null    string        
 7   doi                     548 non-null    string        
 8   type                    548 non-null    string        
 9   journal                 548 non-null    string        
 10  issn                    491 non-null    string        
 11  volume                  480 non-null    string        
 12  issue                   369 non-null    string    

In [ ]:
results, failed = crosref.get_refs(df, concurrent_lim=30)

INFO:artfinder.api:Found 12283 unique references.


KeyboardInterrupt: 

In [ ]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12237 entries, 0 to 0
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   publisher               12237 non-null  string        
 1   license                 7843 non-null   object        
 2   is_referenced_by_count  12237 non-null  object        
 3   link                    11762 non-null  object        
 4   authors                 12237 non-null  object        
 5   abstract                3002 non-null   string        
 6   title                   12201 non-null  string        
 7   doi                     12237 non-null  string        
 8   type                    12237 non-null  string        
 9   journal                 12152 non-null  string        
 10  issn                    12024 non-null  string        
 11  volume                  11885 non-null  string        
 12  issue                   10285 non-null  string        


In [ ]:
articles = results[results['type'].isin(['journal-article', 'proceedings-article'])]

In [ ]:
articles = articles[articles['publication_date'] > '1993']

In [ ]:
articles['title'].fillna('', inplace=True)

/tmp/ipykernel_8102/1427495497.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  articles['title'].fillna('', inplace=True)


In [ ]:
articles.to_csv('database/barcikowski_unfiltered_refs_level1_raw.csv', index=False)

In [ ]:
kaba = load_csv('database/processed/kabashin_full.csv')
barc = load_csv('database/processed/barcikowski_full.csv')

In [ ]:
kaba_barc = pd.concat([kaba, barc], ignore_index=True)

In [ ]:
kaba_barc.drop_duplicates(subset=['doi'], inplace=True)

In [ ]:
strict_refs = articles[articles['title'].map(lambda x: strict_filter(x))]

In [ ]:
strict_refs[~strict_refs['doi'].isin(kaba_barc['doi'])].info()

<class 'pandas.core.frame.DataFrame'>
Index: 869 entries, 0 to 0
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   publisher               869 non-null    string        
 1   license                 568 non-null    object        
 2   is_referenced_by_count  869 non-null    object        
 3   link                    840 non-null    object        
 4   authors                 869 non-null    object        
 5   abstract                174 non-null    string        
 6   title                   869 non-null    string        
 7   doi                     869 non-null    string        
 8   type                    869 non-null    string        
 9   journal                 869 non-null    string        
 10  issn                    869 non-null    string        
 11  volume                  863 non-null    string        
 12  issue                   678 non-null    string        
 1